# Procesamiento de Elecciones de Diputados Federales - Manzanillo (2012-2024)
## Objetivo
El propósito de este notebook es limpiar, estandarizar y estructurar los resultados electorales para las elecciones de Diputados Federales en Manzanillo, abarcando el periodo de 2012 a 2024. El objetivo final es generar archivos de datos limpios y consistentes que puedan ser fácilmente consolidados en una única base de datos para su posterior análisis y visualización geoespacial.

### Manzanillo Diputados Federales 2012

In [1]:
import pandas as pd
import numpy as np
import os

# --- 1. CONFIGURACIÓN DE RUTAS ---
BASE_DIR = '/Users/omartellez/Manzanillo' 

# CAMBIO: Apuntamos a la nueva carpeta
input_folder = os.path.join(BASE_DIR, '01_datos_brutos', 'Diputados_Federales')
output_folder = os.path.join(BASE_DIR, '02_datos_limpios', 'Diputados_Federales')

input_file = 'Manzanillo_DIP_FEDERAL_2012.csv'
output_file = 'Manzanillo_DIP_FEDERAL_2012_limpio.csv'

# --- 2. CARGA DE DATOS ---
file_path = os.path.join(input_folder, input_file)
# Usamos 'utf-8-sig' para manejar la marca BOM (ï»¿)
df = pd.read_csv(file_path, encoding='utf-8-sig')
print("Datos originales de Diputado Federal 2012 cargados.")

# --- 3. LIMPIEZA Y ESTRUCTURACIÓN ---
df_mapped = df.copy()

# a) Estandarizar todos los nombres de columna a minúsculas
df_mapped.columns = df_mapped.columns.str.lower()

# b) Eliminar columnas de contexto y vacías
columnas_a_eliminar = [
    'circunscripcion', 'id_estado', 'nombre_estado', 'id_distrito', 
    'cabecera_distrital', 'id_municipio', 'municipio', 'estatus_acta', 
    'tepjf', 'observaciones', 'ruta_acta'
]
df_mapped = df_mapped.drop(columns=columnas_a_eliminar, errors='ignore')

# c) Renombrar columnas a nuestro formato estándar
mapa_renombre = {
    'nva_alianza': 'na',
    'num_votos_can_nreg': 'voto_no_registrado',
    'num_votos_nulos': 'voto_nulo',
    'total_votos': 'voto_total',
    'lista_nominal': 'lista_nominal' # Aseguramos consistencia
}
df_mapped = df_mapped.rename(columns=mapa_renombre)

# d) Rellenar valores nulos en lista_nominal
if 'lista_nominal' in df_mapped.columns:
    df_mapped['lista_nominal'] = df_mapped['lista_nominal'].fillna(0)

# --- 4. CÁLCULO DE MÉTRICAS ANALÍTICAS (MODELO COMPLETO) ---

# a) Definir competidores y bloques para 2012
competidores_2012 = [
    'pan', 'pri', 'prd', 'pvem', 'pt', 'mc', 'na',
    'pri_pvem', 'prd_pt_mc', 'prd_pt', 'prd_mc', 'pt_mc'
]
proxy_morena_2012 = ['prd', 'pt', 'mc', 'prd_pt_mc', 'prd_pt', 'prd_mc', 'pt_mc']
oposicion_2012 = ['pan', 'pri', 'pvem', 'na', 'pri_pvem']

# b) Cálculo de Bloques (Análisis Macro)
df_mapped['morena_coalitions'] = df_mapped[proxy_morena_2012].sum(axis=1)
df_mapped['non_morena_coalitions'] = df_mapped[oposicion_2012].sum(axis=1)

# c) Composición del Voto "Morena" (Proxy 2012)
df_mapped['voto_morena_solo'] = 0
df_mapped['voto_morena_aliados'] = df_mapped[proxy_morena_2012].sum(axis=1)

# d) Ganador por Casilla
df_competidores = df_mapped[competidores_2012]
df_mapped['ganador_casilla_partido'] = df_competidores.idxmax(axis=1)
df_mapped['ganador_casilla_votos'] = df_competidores.max(axis=1)

# e) Cálculo del Segundo Lugar y Margen
segundo_lugar_votos = np.sort(df_competidores.values, axis=1)[:, -2]
df_mapped['segundo_lugar_votos'] = segundo_lugar_votos
temp_df = df_competidores.copy()
for index, row in temp_df.iterrows():
    winner_col = df_mapped.loc[index, 'ganador_casilla_partido']
    if pd.notna(winner_col) and winner_col in temp_df.columns:
        temp_df.loc[index, winner_col] = np.nan
df_mapped['segundo_lugar_partido'] = temp_df.idxmax(axis=1)
df_mapped['margen_victoria'] = df_mapped['ganador_casilla_votos'] - df_mapped['segundo_lugar_votos']

# --- 5. ORGANIZACIÓN FINAL Y GUARDADO ---
columnas_finales = [
    'seccion', 'casilla', 'lista_nominal',
    # Votos crudos
    'pan', 'pri', 'prd', 'pvem', 'pt', 'mc', 'na', 'pri_pvem', 
    'prd_pt_mc', 'prd_pt', 'prd_mc', 'pt_mc',
    # Analíticas
    'morena_coalitions', 'non_morena_coalitions', 'voto_morena_solo', 'voto_morena_aliados',
    'ganador_casilla_partido', 'ganador_casilla_votos', 'segundo_lugar_partido', 
    'segundo_lugar_votos', 'margen_victoria',
    # Cierre
    'voto_no_registrado', 'voto_nulo', 'voto_total'
]
df_mapped = df_mapped[columnas_finales]

print("\nDatos limpios de 2012 (Diputado Federal) con el modelo de análisis completo:")
df_mapped.info()

# Guardar el archivo limpio
os.makedirs(output_folder, exist_ok=True)
output_path = os.path.join(output_folder, output_file)
df_mapped.to_csv(output_path, index=False)

print(f"\nArchivo limpio y actualizado de 2012 (Diputado Federal) guardado en: {output_path}")

Datos originales de Diputado Federal 2012 cargados.

Datos limpios de 2012 (Diputado Federal) con el modelo de análisis completo:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 27 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   seccion                  200 non-null    int64  
 1   casilla                  200 non-null    object 
 2   lista_nominal            200 non-null    float64
 3   pan                      200 non-null    int64  
 4   pri                      200 non-null    int64  
 5   prd                      200 non-null    int64  
 6   pvem                     200 non-null    int64  
 7   pt                       200 non-null    int64  
 8   mc                       200 non-null    int64  
 9   na                       200 non-null    int64  
 10  pri_pvem                 200 non-null    int64  
 11  prd_pt_mc                200 non-null    int64  
 12  prd_

### Manzanillo Diputados Federales 2015

In [2]:
# --- 1. CONFIGURACIÓN DE RUTAS ---
BASE_DIR = '/Users/omartellez/Manzanillo' 

# CAMBIO: Apuntamos a la nueva carpeta
input_folder = os.path.join(BASE_DIR, '01_datos_brutos', 'Diputados_Federales')
output_folder = os.path.join(BASE_DIR, '02_datos_limpios', 'Diputados_Federales')

input_file = "Manzanillo_DIP_FEDERAL_2015.csv"
output_file = 'Manzanillo_DIP_FEDERAL_2015_limpio.csv'

file_path = os.path.join(input_folder, input_file)
df = pd.read_csv(file_path)

print("Datos originales:")
print(df.info())
df.head(5)

Datos originales:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 216 entries, 0 to 215
Data columns (total 52 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   CIRCUNSCRIPCION     216 non-null    int64  
 1   ID_ESTADO           216 non-null    int64  
 2   NOMBRE_ESTADO       216 non-null    object 
 3   ID_DISTRITO         216 non-null    int64  
 4   CABECERA_DISTRITAL  216 non-null    object 
 5   ID_MUNICIPIO        216 non-null    int64  
 6   MUNICIPIO           216 non-null    object 
 7   SECCION             216 non-null    int64  
 8   CASILLA             216 non-null    object 
 9   PAN                 216 non-null    int64  
 10  PRI                 216 non-null    int64  
 11  PRD                 216 non-null    int64  
 12  PVEM                216 non-null    int64  
 13  PT                  216 non-null    int64  
 14  MC                  216 non-null    int64  
 15  NVA_ALIANZA         216 non-null    int

,CIRCUNSCRIPCION,ID_ESTADO,NOMBRE_ESTADO,ID_DISTRITO,CABECERA_DISTRITAL,ID_MUNICIPIO,MUNICIPIO,SECCION,CASILLA,PAN,...,CAND_IND21,CAND_IND22,NUM_VOTOS_CAN_NREG,NUM_VOTOS_NULOS,TOTAL_VOTOS,LISTA_NOMINAL,ESTATUS_ACTA,TEPJF,OBSERVACIONES,RUTA_ACTA
0,5,6,COLIMA,2,MANZANILLO,8,MANZANILLO,200,B,146,...,NaN,NaN,0,10,349,619,ACTA DE CASILLA,NaN,NaN,JE2015_06_02_0200_1_B_0_2.jpg
1,5,6,COLIMA,2,MANZANILLO,8,MANZANILLO,201,B,34,...,NaN,NaN,0,2,83,129,RECUENTO(SRA),NaN,NaN,JE2015_06_02_6_GPO1.pdf
2,5,6,COLIMA,2,MANZANILLO,8,MANZANILLO,202,B,157,...,NaN,NaN,0,13,337,620,ACTA DE CASILLA,NaN,NaN,JE2015_06_02_0202_1_B_0_2.jpg
3,5,6,COLIMA,2,MANZANILLO,8,MANZANILLO,202,C01,152,...,NaN,NaN,0,14,344,620,ACTA DE CASILLA,NaN,NaN,JE2015_06_02_0202_1_C_0_2.jpg
4,5,6,COLIMA,2,MANZANILLO,8,MANZANILLO,202,C02,150,...,NaN,NaN,0,7,348,620,ACTA DE CASILLA,NaN,NaN,JE2015_06_02_0202_2_C_0_2.jpg


In [3]:
# --- 3. LIMPIEZA Y ESTRUCTURACIÓN ---
df_mapped = df.copy()

# a) Estandarizar todos los nombres de columna a minúsculas
df_mapped.columns = df_mapped.columns.str.lower()

# b) Eliminar columnas de contexto, vacías y casi vacías
# Generamos la lista de todas las columnas de candidatos independientes para eliminar
cand_ind_cols = [f'cand_ind{i}' for i in range(1, 23)]
columnas_a_eliminar = [
    'circunscripcion', 'id_estado', 'nombre_estado', 'id_distrito', 
    'cabecera_distrital', 'id_municipio', 'municipio', 'estatus_acta', 
    'tepjf', 'observaciones', 'ruta_acta', 'pan_nva_alianza' # Columna vacía
]
columnas_a_eliminar.extend(cand_ind_cols) # Añadimos todos los CAND_IND a la lista
df_mapped = df_mapped.drop(columns=columnas_a_eliminar, errors='ignore')

# c) Renombrar columnas a nuestro formato estándar
mapa_renombre = {
    'casilla': 'casilla', # 'CASILLA' ya pasó a minúsculas
    'seccion': 'seccion', # 'SECCION' ya pasó a minúsculas
    'nva_alianza': 'na',
    'num_votos_can_nreg': 'voto_no_registrado',
    'num_votos_nulos': 'voto_nulo',
    'total_votos': 'voto_total',
    'lista_nominal': 'lista_nominal'
}
df_mapped = df_mapped.rename(columns=mapa_renombre)

# --- 4. CÁLCULO DE MÉTRICAS ANALÍTICAS (MODELO COMPLETO) ---
# a) Definir competidores y bloques para 2015
competidores_2015 = [
    'pan', 'pri', 'prd', 'pvem', 'pt', 'mc', 'na', 'morena', 'ph', 'es',
    'pri_pvem', 'prd_pt'
]
partidos_morena_2015 = ['morena'] # Morena compitió solo
oposicion_2015 = [col for col in competidores_2015 if col != 'morena']

# b) Cálculo de Bloques (Análisis Macro)
df_mapped['morena_coalitions'] = df_mapped[partidos_morena_2015].sum(axis=1)
df_mapped['non_morena_coalitions'] = df_mapped[oposicion_2015].sum(axis=1)

# c) Composición del Voto "Morena"
df_mapped['voto_morena_solo'] = df_mapped['morena']
df_mapped['voto_morena_aliados'] = 0

# d) Ganador por Casilla
df_competidores = df_mapped[competidores_2015]
df_mapped['ganador_casilla_partido'] = df_competidores.idxmax(axis=1)
df_mapped['ganador_casilla_votos'] = df_competidores.max(axis=1)

# e) Cálculo del Segundo Lugar y Margen
segundo_lugar_votos = np.sort(df_competidores.values, axis=1)[:, -2]
df_mapped['segundo_lugar_votos'] = segundo_lugar_votos
temp_df = df_competidores.copy()
for index, row in temp_df.iterrows():
    winner_col = df_mapped.loc[index, 'ganador_casilla_partido']
    if pd.notna(winner_col) and winner_col in temp_df.columns:
        temp_df.loc[index, winner_col] = np.nan
df_mapped['segundo_lugar_partido'] = temp_df.idxmax(axis=1)
df_mapped['margen_victoria'] = df_mapped['ganador_casilla_votos'] - df_mapped['segundo_lugar_votos']

# --- 5. ORGANIZACIÓN FINAL Y GUARDADO ---
columnas_finales = [
    'seccion', 'casilla', 'lista_nominal',
    # Votos crudos
    'pan', 'pri', 'prd', 'pvem', 'pt', 'mc', 'na', 'morena', 'ph', 'es',
    'pri_pvem', 'prd_pt',
    # Analíticas
    'morena_coalitions', 'non_morena_coalitions', 'voto_morena_solo', 'voto_morena_aliados',
    'ganador_casilla_partido', 'ganador_casilla_votos', 'segundo_lugar_partido', 
    'segundo_lugar_votos', 'margen_victoria',
    # Cierre
    'voto_no_registrado', 'voto_nulo', 'voto_total'
]
df_mapped = df_mapped[columnas_finales]

print("\nDatos limpios de 2015 (Diputado Federal) con el modelo de análisis completo:")
df_mapped.info()

# Guardar el archivo limpio
os.makedirs(output_folder, exist_ok=True)
output_path = os.path.join(output_folder, output_file)
df_mapped.to_csv(output_path, index=False)
print(f"\nArchivo limpio y actualizado de 2015 (Diputado Federal) guardado en: {output_path}")



Datos limpios de 2015 (Diputado Federal) con el modelo de análisis completo:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 216 entries, 0 to 215
Data columns (total 27 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   seccion                  216 non-null    int64 
 1   casilla                  216 non-null    object
 2   lista_nominal            216 non-null    int64 
 3   pan                      216 non-null    int64 
 4   pri                      216 non-null    int64 
 5   prd                      216 non-null    int64 
 6   pvem                     216 non-null    int64 
 7   pt                       216 non-null    int64 
 8   mc                       216 non-null    int64 
 9   na                       216 non-null    int64 
 10  morena                   216 non-null    int64 
 11  ph                       216 non-null    int64 
 12  es                       216 non-null    int64 
 13  pri_pvem         

### Manzanillo Diputados 2018

In [4]:
input_file = "Manzanillo_DIP_FEDERAL_2018.csv"
output_file = 'Manzanillo_DIP_FEDERAL_2018_limpio.csv'

file_path = os.path.join(input_folder, input_file)
df = pd.read_csv(file_path)

print("Datos originales:")
print(df.info())
df.head(5)

Datos originales:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 233 entries, 0 to 232
Data columns (total 80 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   CIRCUNSCRIPCION     233 non-null    int64  
 1   ID_ESTADO           233 non-null    int64  
 2   NOMBRE_ESTADO       233 non-null    object 
 3   ID_DISTRITO         233 non-null    int64  
 4   CABECERA_DISTRITAL  233 non-null    object 
 5   ID_MUNICIPIO        233 non-null    int64  
 6   MUNICIPIO           233 non-null    object 
 7   SECCION             233 non-null    int64  
 8   TIPO_CASILLA        233 non-null    object 
 9   ID_CASILLA          233 non-null    int64  
 10  EXT_CONTIGUA        233 non-null    int64  
 11  CASILLA             233 non-null    object 
 12  PAN                 233 non-null    int64  
 13  PRI                 233 non-null    int64  
 14  PRD                 233 non-null    int64  
 15  PVEM                233 non-null    int

,CIRCUNSCRIPCION,ID_ESTADO,NOMBRE_ESTADO,ID_DISTRITO,CABECERA_DISTRITAL,ID_MUNICIPIO,MUNICIPIO,SECCION,TIPO_CASILLA,ID_CASILLA,...,CAND_IND38,NUM_VOTOS_CAN_NREG,NUM_VOTOS_NULOS,TOTAL_VOTOS,LISTA_NOMINAL,ESTATUS_ACTA,TRIBUNAL,JUICIO,OBSERVACIONES,RUTA_ACTA
0,5,6,COLIMA,2,MANZANILLO,8,MANZANILLO,200,B,1,...,NaN,0,6,381,606,Recuento,NaN,NaN,NaN,JE2018_06_02_DIP_GPO1P.pdf
1,5,6,COLIMA,2,MANZANILLO,8,MANZANILLO,201,B,1,...,NaN,0,5,77,123,Recuento,NaN,NaN,NaN,JE2018_06_02_DIP_GPO1P.pdf
2,5,6,COLIMA,2,MANZANILLO,8,MANZANILLO,202,B,1,...,NaN,0,12,356,632,Recuento,NaN,NaN,NaN,JE2018_06_02_DIP_VR.pdf
3,5,6,COLIMA,2,MANZANILLO,8,MANZANILLO,202,C,1,...,NaN,0,22,381,632,Cotejo,NaN,NaN,NaN,JE2018_06_02_DIP_202_1_C_0_12977.jpg
4,5,6,COLIMA,2,MANZANILLO,8,MANZANILLO,202,C,2,...,NaN,0,17,387,632,Recuento,NaN,NaN,NaN,JE2018_06_02_DIP_GPO1P.pdf


In [5]:
# --- 3. LIMPIEZA Y ESTRUCTURACIÓN ---
df_mapped = df.copy()

# a) Estandarizar todos los nombres de columna a minúsculas
df_mapped.columns = df_mapped.columns.str.lower()

# b) Eliminar columnas de contexto, vacías y casi vacías
cand_ind_cols = [f'cand_ind{i}' for i in range(1, 39)]
columnas_a_eliminar = [
    'circunscripcion', 'id_estado', 'nombre_estado', 'id_distrito', 
    'cabecera_distrital', 'id_municipio', 'municipio', 'id_casilla', 
    'ext_contigua', 'tipo_casilla', # Nos quedamos con 'casilla' que es más limpia
    'estatus_acta', 'tribunal', 'juicio', 'observaciones', 'ruta_acta'
]
columnas_a_eliminar.extend(cand_ind_cols)
df_mapped = df_mapped.drop(columns=columnas_a_eliminar, errors='ignore')

# c) Renombrar columnas a nuestro formato estándar
mapa_renombre = {
    'num_votos_can_nreg': 'voto_no_registrado',
    'num_votos_nulos': 'voto_nulo',
    'total_votos': 'voto_total',
    'lista_nominal': 'lista_nominal'
}
df_mapped = df_mapped.rename(columns=mapa_renombre)

# --- 4. CÁLCULO DE MÉTRICAS ANALÍTICAS (MODELO COMPLETO) ---
# a) Definir competidores y bloques para 2018
competidores_2018 = [
    'pan', 'pri', 'prd', 'pvem', 'pt', 'mc', 'na', 'morena', 'es',
    'pan_prd_mc', 'pan_prd', 'pan_mc', 'prd_mc',
    'pri_pvem_na', 'pri_pvem', 'pri_na', 'pvem_na',
    'pt_morena_es', 'pt_morena', 'pt_es', 'morena_es'
]
# Nos aseguramos de usar solo las columnas que existen en el DF
competidores_2018 = [col for col in competidores_2018 if col in df_mapped.columns]

partidos_morena_2018_total = [col for col in ['morena', 'pt', 'es', 'pt_morena_es', 'pt_morena', 'pt_es', 'morena_es'] if col in df_mapped.columns]
partidos_morena_2018_aliados = [col for col in ['pt', 'es', 'pt_morena_es', 'pt_morena', 'pt_es', 'morena_es'] if col in df_mapped.columns]
oposicion_2018 = [col for col in competidores_2018 if col not in partidos_morena_2018_total]

# b) Cálculo de Bloques (Análisis Macro)
df_mapped['morena_coalitions'] = df_mapped[partidos_morena_2018_total].sum(axis=1)
df_mapped['non_morena_coalitions'] = df_mapped[oposicion_2018].sum(axis=1)

# c) Composición del Voto "Morena"
df_mapped['voto_morena_solo'] = df_mapped['morena']
df_mapped['voto_morena_aliados'] = df_mapped[partidos_morena_2018_aliados].sum(axis=1)

# d) Ganador por Casilla
df_competidores = df_mapped[competidores_2018]
df_mapped['ganador_casilla_partido'] = df_competidores.idxmax(axis=1)
df_mapped['ganador_casilla_votos'] = df_competidores.max(axis=1)

# e) Cálculo del Segundo Lugar y Margen
segundo_lugar_votos = np.sort(df_competidores.values, axis=1)[:, -2]
df_mapped['segundo_lugar_votos'] = segundo_lugar_votos
temp_df = df_competidores.copy()
for index, row in temp_df.iterrows():
    winner_col = df_mapped.loc[index, 'ganador_casilla_partido']
    if pd.notna(winner_col) and winner_col in temp_df.columns:
        temp_df.loc[index, winner_col] = np.nan
df_mapped['segundo_lugar_partido'] = temp_df.idxmax(axis=1)
df_mapped['margen_victoria'] = df_mapped['ganador_casilla_votos'] - df_mapped['segundo_lugar_votos']

# --- 5. ORGANIZACIÓN FINAL Y GUARDADO ---
columnas_finales = [
    'seccion', 'casilla', 'lista_nominal',
    # Votos crudos
    'pan', 'pri', 'prd', 'pvem', 'pt', 'mc', 'na', 'morena', 'es',
    'pan_prd_mc', 'pan_prd', 'pan_mc', 'prd_mc',
    'pri_pvem_na', 'pri_pvem', 'pri_na', 'pvem_na',
    'pt_morena_es', 'pt_morena', 'pt_es', 'morena_es',
    # Analíticas
    'morena_coalitions', 'non_morena_coalitions', 'voto_morena_solo', 'voto_morena_aliados',
    'ganador_casilla_partido', 'ganador_casilla_votos', 'segundo_lugar_partido', 
    'segundo_lugar_votos', 'margen_victoria',
    # Cierre
    'voto_no_registrado', 'voto_nulo', 'voto_total'
]
# Ordenamos solo con las columnas que existen para evitar errores
columnas_existentes = [col for col in columnas_finales if col in df_mapped.columns]
df_mapped = df_mapped[columnas_existentes]

print("\nDatos limpios de 2018 (Diputado Federal) con el modelo de análisis completo:")
df_mapped.info()

# Guardar el archivo limpio
os.makedirs(output_folder, exist_ok=True)
output_path = os.path.join(output_folder, output_file)
df_mapped.to_csv(output_path, index=False)
print(f"\nArchivo limpio y actualizado de 2018 (Diputado Federal) guardado en: {output_path}")



Datos limpios de 2018 (Diputado Federal) con el modelo de análisis completo:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 233 entries, 0 to 232
Data columns (total 36 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   seccion                  233 non-null    int64 
 1   casilla                  233 non-null    object
 2   lista_nominal            233 non-null    int64 
 3   pan                      233 non-null    int64 
 4   pri                      233 non-null    int64 
 5   prd                      233 non-null    int64 
 6   pvem                     233 non-null    int64 
 7   pt                       233 non-null    int64 
 8   mc                       233 non-null    int64 
 9   na                       233 non-null    int64 
 10  morena                   233 non-null    int64 
 11  es                       233 non-null    int64 
 12  pan_prd_mc               233 non-null    int64 
 13  pan_prd          

### Manzanillo Ayuntamientos 2021

In [6]:
input_file = 'Manzanillo_DIP_FEDERAL_2021.csv' 
output_file = 'Manzanillo_DIP_FEDERAL_2021_limpio.csv'

# --- 2. CARGA DE DATOS ---
file_path = os.path.join(input_folder, input_file)
df = pd.read_csv(file_path, encoding='utf-8-sig', low_memory=False)
print("Datos originales de Diputado Federal 2021 cargados.")

# --- 3. LIMPIEZA Y ESTRUCTURACIÓN ---
df_mapped = df.copy()

# a) Estandarizar todos los nombres de columna a minúsculas
df_mapped.columns = df_mapped.columns.str.lower()

# b) Eliminar columnas de contexto, vacías y redundantes
cand_ind_cols = [f'cand_ind{i}' for i in range(1, 4)]
columnas_a_eliminar = [
    'circunscripcion', 'id_estado', 'nombre_estado', 'id_distrito_federal', 
    'cabecera_distrital_federal', 'id_municipio', 'municipio', 'id_casilla', 
    'ext_contigua', 'tipo_casilla', 'voto_valido', 'estatus_acta', 'tribunal', 'juicio'
]
columnas_a_eliminar.extend(cand_ind_cols)
df_mapped = df_mapped.drop(columns=columnas_a_eliminar, errors='ignore')

# c) Renombrar columnas a nuestro formato estándar
mapa_renombre = {
    'num_votos_can_nreg': 'voto_no_registrado',
    'num_votos_nulos': 'voto_nulo',
    'total_votos': 'voto_total',
    'pes': 'es' # Partido Encuentro Solidario, para mantener consistencia con 'es' de 2018
}
df_mapped = df_mapped.rename(columns=mapa_renombre)

# --- 4. CÁLCULO DE MÉTRICAS ANALÍTICAS (MODELO COMPLETO) ---
# a) Definir competidores y bloques para 2021
competidores_2021 = [
    'pan', 'pri', 'prd', 'pvem', 'pt', 'mc', 'morena', 'es', 'rsp', 'fxm',
    'pan_pri_prd', 'pan_pri', 'pan_prd', 'pri_prd',
    'pvem_pt_morena', 'pvem_pt', 'pvem_morena', 'pt_morena'
]
competidores_2021 = [col for col in competidores_2021 if col in df_mapped.columns]

partidos_morena_2021_total = [col for col in ['morena', 'pvem', 'pt', 'pvem_pt_morena', 'pvem_pt', 'pvem_morena', 'pt_morena'] if col in df_mapped.columns]
partidos_morena_2021_aliados = [col for col in ['pvem', 'pt', 'pvem_pt_morena', 'pvem_pt', 'pvem_morena', 'pt_morena'] if col in df_mapped.columns]
oposicion_2021 = [col for col in competidores_2021 if col not in partidos_morena_2021_total]

# b) Cálculo de Bloques (Análisis Macro)
df_mapped['morena_coalitions'] = df_mapped[partidos_morena_2021_total].sum(axis=1)
df_mapped['non_morena_coalitions'] = df_mapped[oposicion_2021].sum(axis=1)

# c) Composición del Voto "Morena"
df_mapped['voto_morena_solo'] = df_mapped['morena']
df_mapped['voto_morena_aliados'] = df_mapped[partidos_morena_2021_aliados].sum(axis=1)

# d) Ganador por Casilla
df_competidores = df_mapped[competidores_2021]
df_mapped['ganador_casilla_partido'] = df_competidores.idxmax(axis=1)
df_mapped['ganador_casilla_votos'] = df_competidores.max(axis=1)

# e) Cálculo del Segundo Lugar y Margen
segundo_lugar_votos = np.sort(df_competidores.values, axis=1)[:, -2]
df_mapped['segundo_lugar_votos'] = segundo_lugar_votos
temp_df = df_competidores.copy()
for index, row in temp_df.iterrows():
    winner_col = df_mapped.loc[index, 'ganador_casilla_partido']
    if pd.notna(winner_col) and winner_col in temp_df.columns:
        temp_df.loc[index, winner_col] = np.nan
df_mapped['segundo_lugar_partido'] = temp_df.idxmax(axis=1)
df_mapped['margen_victoria'] = df_mapped['ganador_casilla_votos'] - df_mapped['segundo_lugar_votos']

# --- 5. ORGANIZACIÓN FINAL Y GUARDADO ---
columnas_finales = [
    'seccion', 'casilla', 'lista_nominal',
    # Votos crudos
    'pan', 'pri', 'prd', 'pvem', 'pt', 'mc', 'morena', 'es', 'rsp', 'fxm',
    'pan_pri_prd', 'pan_pri', 'pan_prd', 'pri_prd',
    'pvem_pt_morena', 'pvem_pt', 'pvem_morena', 'pt_morena',
    # Analíticas
    'morena_coalitions', 'non_morena_coalitions', 'voto_morena_solo', 'voto_morena_aliados',
    'ganador_casilla_partido', 'ganador_casilla_votos', 'segundo_lugar_partido', 
    'segundo_lugar_votos', 'margen_victoria',
    # Cierre
    'voto_no_registrado', 'voto_nulo', 'voto_total'
]
df_mapped = df_mapped[columnas_finales]

print("\nDatos limpios de 2021 (Diputado Federal) con el modelo de análisis completo:")
df_mapped.info()

# Guardar el archivo limpio
os.makedirs(output_folder, exist_ok=True)
output_path = os.path.join(output_folder, output_file)
df_mapped.to_csv(output_path, index=False)
print(f"\nArchivo limpio y actualizado de 2021 (Diputado Federal) guardado en: {output_path}")

Datos originales de Diputado Federal 2021 cargados.

Datos limpios de 2021 (Diputado Federal) con el modelo de análisis completo:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 239 entries, 0 to 238
Data columns (total 33 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   seccion                  239 non-null    int64 
 1   casilla                  239 non-null    object
 2   lista_nominal            239 non-null    int64 
 3   pan                      239 non-null    int64 
 4   pri                      239 non-null    int64 
 5   prd                      239 non-null    int64 
 6   pvem                     239 non-null    int64 
 7   pt                       239 non-null    int64 
 8   mc                       239 non-null    int64 
 9   morena                   239 non-null    int64 
 10  es                       239 non-null    int64 
 11  rsp                      239 non-null    int64 
 12  fxm               

### Manzanillo Diputado Federal 2024

In [7]:
input_file = 'Manzanillo_DIP_FEDERAL_2024.csv' 
output_file = 'Manzanillo_DIP_FEDERAL_2024_limpio.csv'

# --- 2. CARGA DE DATOS ---
file_path = os.path.join(input_folder, input_file)
df = pd.read_csv(file_path, encoding='utf-8-sig', low_memory=False)
print("Datos originales de Diputado Federal 2024 cargados.")

df.info()


Datos originales de Diputado Federal 2024 cargados.
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 220 entries, 0 to 219
Data columns (total 35 columns):
 #   Column                       Non-Null Count  Dtype 
---  ------                       --------------  ----- 
 0   CLAVE_CASILLA                220 non-null    object
 1   CLAVE_ACTA                   220 non-null    object
 2   ID_ENTIDAD                   220 non-null    int64 
 3   ENTIDAD                      220 non-null    object
 4   ID_DISTRITO_FEDERAL          220 non-null    int64 
 5   DISTRITO_FEDERAL             220 non-null    object
 6   SECCION                      220 non-null    int64 
 7   ID_CASILLA                   220 non-null    int64 
 8   TIPO_CASILLA                 220 non-null    object
 9   EXT_CONTIGUA                 220 non-null    int64 
 10  CASILLA                      220 non-null    object
 11  TIPO_ACTA                    220 non-null    object
 12  PAN                          220 non-nul

In [9]:
import re
# --- 3. LIMPIEZA Y ESTRUCTURACIÓN (VERSIÓN CORREGIDA PARA CASILLA) ---
df_mapped = df.copy()

# a) Limpieza robusta de nombres de columna
def limpiar_nombre_columna(col_name):
    clean_col = col_name.lower()
    clean_col = re.sub(r'[/ ]', '_', clean_col)
    clean_col = re.sub(r'[^a-z0-9_]', '', clean_col)
    return clean_col.strip('_')
df_mapped.columns = [limpiar_nombre_columna(col) for col in df_mapped.columns]

# b) CREACIÓN DE LA COLUMNA 'casilla' ESTANDARIZADA (ANTES DE BORRAR NADA)
if 'tipo_casilla' in df_mapped.columns and 'id_casilla' in df_mapped.columns:
    df_mapped['id_casilla'] = df_mapped['id_casilla'].astype(str)
    # Concatenamos para crear el identificador, ej: 'B' + '1' -> 'B1'
    df_mapped['casilla_final'] = df_mapped['tipo_casilla'].str.strip() + df_mapped['id_casilla']
    print("     -> Columna 'casilla' estandarizada creada exitosamente.")
else:
    df_mapped['casilla_final'] = 'N/A' # Fallback por si acaso

# c) Eliminar columnas de contexto y redundantes
columnas_a_eliminar = [
    'clave_casilla', 'clave_acta', 'id_entidad', 'entidad', 'id_distrito_federal', 
    'distrito_federal', 'ext_contigua', 'tipo_acta', 'observaciones',
    'mecanismos_traslado', 'fecha_hora', 
    'id_casilla',       # La original que ya usamos
    'tipo_casilla',     # La original que ya usamos
    'casilla'           # La original con 'Urbana'/'No Urbana'
]
df_mapped = df_mapped.drop(columns=columnas_a_eliminar, errors='ignore')

# d) Renombrar la nueva columna a su nombre final
df_mapped = df_mapped.rename(columns={'casilla_final': 'casilla'})

# e) Renombrar el resto de columnas a nuestro formato estándar
mapa_renombre = {
    'candidato_a_independiente': 'independiente',
    'candidato_a_no_registrado_a': 'voto_no_registrado',
    'votos_nulos': 'voto_nulo',
    'total_votos_calculados': 'voto_total'
}
df_mapped = df_mapped.rename(columns=mapa_renombre)

# f) CORREGIR TIPOS DE DATO (reemplazar '-')
columnas_a_convertir = [
    'independiente', 'pan_pri_prd', 'pan_pri', 'pan_prd', 'pri_prd',
    'pvem_pt_morena', 'pvem_pt', 'pvem_morena', 'pt_morena'
]
for col in columnas_a_convertir:
    if col in df_mapped.columns:
        if df_mapped[col].dtype == 'object':
            df_mapped[col] = df_mapped[col].replace('-', '0', regex=False)
        df_mapped[col] = pd.to_numeric(df_mapped[col], errors='coerce')

df_mapped = df_mapped.fillna(0)

# --- 4. CÁLCULO DE MÉTRICAS ANALÍTICAS ---
competidores_2024 = [
    'pan', 'pri', 'prd', 'pvem', 'pt', 'mc', 'morena', 'independiente',
    'pan_pri_prd', 'pan_pri', 'pan_prd', 'pri_prd',
    'pvem_pt_morena', 'pvem_pt', 'pvem_morena', 'pt_morena'
]
competidores_2024 = [col for col in competidores_2024 if col in df_mapped.columns]
partidos_morena_2024_total = [col for col in ['morena', 'pvem', 'pt', 'pvem_pt_morena', 'pvem_pt', 'pvem_morena', 'pt_morena'] if col in df_mapped.columns]
partidos_morena_2024_aliados = [col for col in ['pvem', 'pt', 'pvem_pt_morena', 'pvem_pt', 'pvem_morena', 'pt_morena'] if col in df_mapped.columns]
oposicion_2024 = [col for col in competidores_2024 if col not in partidos_morena_2024_total]

# El resto de la lógica de cálculo es la estándar y correcta
df_mapped['morena_coalitions'] = df_mapped[partidos_morena_2024_total].sum(axis=1)
df_mapped['non_morena_coalitions'] = df_mapped[oposicion_2024].sum(axis=1)
df_mapped['voto_morena_solo'] = df_mapped['morena']
df_mapped['voto_morena_aliados'] = df_mapped[partidos_morena_2024_aliados].sum(axis=1)
df_competidores = df_mapped[competidores_2024]
df_mapped['ganador_casilla_partido'] = df_competidores.idxmax(axis=1)
df_mapped['ganador_casilla_votos'] = df_competidores.max(axis=1)
if len(df_competidores.columns) > 1:
    segundo_lugar_votos = np.sort(df_competidores.values, axis=1)[:, -2]
    df_mapped['segundo_lugar_votos'] = segundo_lugar_votos
    temp_df = df_competidores.copy()
    for index, row in temp_df.iterrows():
        winner_col = df_mapped.loc[index, 'ganador_casilla_partido']
        if pd.notna(winner_col) and winner_col in temp_df.columns:
            temp_df.loc[index, winner_col] = np.nan
    df_mapped['segundo_lugar_partido'] = temp_df.idxmax(axis=1)
    df_mapped['margen_victoria'] = df_mapped['ganador_casilla_votos'] - df_mapped['segundo_lugar_votos']
else:
    df_mapped['segundo_lugar_votos'] = 0
    df_mapped['segundo_lugar_partido'] = "N/A"
    df_mapped['margen_victoria'] = df_mapped['ganador_casilla_votos']

# --- 5. ORGANIZACIÓN FINAL Y GUARDADO ---
columnas_finales = [
    'seccion', 'casilla', 'lista_nominal',
    'pan', 'pri', 'prd', 'pvem', 'pt', 'mc', 'morena', 'independiente',
    'pan_pri_prd', 'pan_pri', 'pan_prd', 'pri_prd',
    'pvem_pt_morena', 'pvem_pt', 'pvem_morena', 'pt_morena',
    'morena_coalitions', 'non_morena_coalitions', 'voto_morena_solo', 'voto_morena_aliados',
    'ganador_casilla_partido', 'ganador_casilla_votos', 'segundo_lugar_partido', 
    'segundo_lugar_votos', 'margen_victoria',
    'voto_no_registrado', 'voto_nulo', 'voto_total'
]
# Hacemos el reordenamiento a prueba de errores
columnas_existentes = [col for col in columnas_finales if col in df_mapped.columns]
df_mapped = df_mapped[columnas_existentes]

print("\nDatos limpios de 2024 (diputado federal) con el modelo de análisis completo:")
df_mapped.info()

# Guardar el archivo limpio
os.makedirs(output_folder, exist_ok=True)
output_path = os.path.join(output_folder, output_file)
df_mapped.to_csv(output_path, index=False)
print(f"\nArchivo limpio y actualizado de 2024 (Diputado Federal) guardado en: {output_path}")

     -> Columna 'casilla' estandarizada creada exitosamente.

Datos limpios de 2024 (diputado federal) con el modelo de análisis completo:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 220 entries, 0 to 219
Data columns (total 31 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   seccion                  220 non-null    int64 
 1   casilla                  220 non-null    object
 2   lista_nominal            220 non-null    int64 
 3   pan                      220 non-null    int64 
 4   pri                      220 non-null    int64 
 5   prd                      220 non-null    int64 
 6   pvem                     220 non-null    int64 
 7   pt                       220 non-null    int64 
 8   mc                       220 non-null    int64 
 9   morena                   220 non-null    int64 
 10  independiente            220 non-null    int64 
 11  pan_pri_prd              220 non-null    int64 
 12  pan_pri  

## Base consolidada elecciones Diputados en Manzanillo 2012-2024

In [10]:
import pandas as pd
import os
import glob

# --- 1. CONFIGURACIÓN ---
BASE_DIR = '/Users/omartellez/Manzanillo'

# Apuntamos a la carpeta de Diputados Federales limpios
input_folder = os.path.join(BASE_DIR, '02_datos_limpios', 'Diputados_Federales') 
output_folder = os.path.join(BASE_DIR, '03_base_consolidada')
output_file = 'diputado_federal_consolidado_2012_2024.csv'

# --- 2. ENCONTRAR Y CARGAR TODOS LOS ARCHIVOS LIMPIOS ---
search_pattern = os.path.join(input_folder, '*_limpio.csv')
clean_files = sorted(glob.glob(search_pattern))

if not clean_files:
    print("No se encontraron archivos limpios de Diputado Federal para consolidar.")
else:
    all_dfs = []
    print(f"Se encontraron {len(clean_files)} archivos para consolidar:")
    for file_path in clean_files:
        print(f"  -> Cargando: {os.path.basename(file_path)}")

        year = int(os.path.basename(file_path).split('_')[-2])
        df = pd.read_csv(file_path)
        df['anio'] = year

        # --- LÍNEA CLAVE Y CORREGIDA ---
        df['tipo_eleccion'] = 'Diputado_Federal'

        all_dfs.append(df)

    # --- 3. CONSOLIDAR, ORDENAR Y GUARDAR ---
    consolidated_df = pd.concat(all_dfs, ignore_index=True)

    # Lógica de ordenamiento automático y robusto
    columnas_inicio = [
        'seccion', 'casilla', 'anio', 'tipo_eleccion', 'lista_nominal',
        'ganador_casilla_partido', 'ganador_casilla_votos', 'segundo_lugar_partido', 
        'segundo_lugar_votos', 'margen_victoria',
        'morena_coalitions', 'non_morena_coalitions', 'voto_morena_solo', 'voto_morena_aliados'
    ]
    columnas_fin = ['voto_no_registrado', 'voto_nulo', 'voto_total']

    columnas_votos_crudos = sorted([
        col for col in consolidated_df.columns 
        if col not in columnas_inicio and col not in columnas_fin
    ])

    orden_final = columnas_inicio + columnas_votos_crudos + columnas_fin

    # Asegurar que solo se usan columnas que existen
    orden_existente = [col for col in orden_final if col in consolidated_df.columns]
    consolidated_df_ordenado = consolidated_df[orden_existente]
    consolidated_df_final = consolidated_df_ordenado.fillna(0)

    # Guardar la base de datos consolidada
    os.makedirs(output_folder, exist_ok=True)
    output_path = os.path.join(output_folder, output_file)
    consolidated_df_final.to_csv(output_path, index=False)

    print("\n¡Base de datos de Diputado Federal consolidada y ordenada creada exitosamente!")
    print(f"Guardada en: {output_path}")
    print("\nResumen de la base consolidada final:")
    consolidated_df_final.info()

Se encontraron 5 archivos para consolidar:
  -> Cargando: Manzanillo_DIP_FEDERAL_2012_limpio.csv
  -> Cargando: Manzanillo_DIP_FEDERAL_2015_limpio.csv
  -> Cargando: Manzanillo_DIP_FEDERAL_2018_limpio.csv
  -> Cargando: Manzanillo_DIP_FEDERAL_2021_limpio.csv
  -> Cargando: Manzanillo_DIP_FEDERAL_2024_limpio.csv

¡Base de datos de Diputado Federal consolidada y ordenada creada exitosamente!
Guardada en: /Users/omartellez/Manzanillo/03_base_consolidada/diputado_federal_consolidado_2012_2024.csv

Resumen de la base consolidada final:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1108 entries, 0 to 1107
Data columns (total 51 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   seccion                  1108 non-null   int64  
 1   casilla                  1108 non-null   object 
 2   anio                     1108 non-null   int64  
 3   tipo_eleccion            1108 non-null   object 
 4   lista_nominal            11